# 如何从工具返回工件

:::info 前置条件
本指南假设您已熟悉以下概念：

- [ToolMessage](/docs/concepts/messages/#toolmessage)
- [工具](/docs/concepts/tools)
- [函数/工具调用](/docs/concepts/tool_calling)

:::

[工具](/docs/concepts/tools/) 是可以被 [模型调用](/docs/concepts/tool_calling/) 的实用程序，其输出旨在反馈给模型。然而，有时我们希望将工具执行的工件提供给链或代理中的下游组件，但不希望将其暴露给模型本身。例如，如果工具返回一个自定义对象、数据框或图像，我们可能希望将有关此输出的一些元数据传递给模型，而不是将实际输出传递给模型。同时，我们可能希望能够在其他地方（例如下游工具中）访问此完整输出。

Tool 和 [ToolMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolMessage.html) 接口使我们能够区分工具输出中供模型使用的部分（这是 ToolMessage.content）和供模型外部使用的部分（ToolMessage.artifact）。

:::info 需要 ``langchain-core >= 0.2.19``

此功能已添加到 ``langchain-core == 0.2.19``。请确保您的软件包是最新的。

:::

## 定义工具

如果我们希望工具区分消息内容和其他工件，则需要在定义工具时指定 `response_format="content_and_artifact"`，并确保返回一个 (content, artifact) 元组：

In [ ]:
%pip install -qU "langchain-core>=0.2.19"

In [ ]:
import random
from typing import List, Tuple

from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def generate_random_ints(min: int, max: int, size: int) -> Tuple[str, List[int]]:
    """生成范围 [min, max] 内的 size 个随机整数。"""
    array = [random.randint(min, max) for _ in range(size)]
    content = f"成功生成了一个包含 {size} 个随机整数的数组，范围为 [{min}, {max}]。"
    return content, array

## 使用 ToolCall 调用工具

如果我们仅使用工具参数直接调用工具，您会注意到我们只会得到工具输出的内容部分：

In [3]:
generate_random_ints.invoke({"min": 0, "max": 9, "size": 10})

'Successfully generated array of 10 random ints in [0, 9].'

为了同时获取内容和工件，我们需要使用 ToolCall 调用模型（ToolCall 只是一个包含 "name"、"args"、"id" 和 "type" 键的字典），它包含生成 ToolMessage 所需的附加信息，例如工具调用 ID：

In [ ]:
generate_random_ints.invoke(
    {
        "name": "generate_random_ints",
        "args": {"min": 0, "max": 9, "size": 10},
        "id": "123",  # 必需
        "type": "tool_call",  # 必需
    }
)

ToolMessage(content='Successfully generated array of 10 random ints in [0, 9].', name='generate_random_ints', tool_call_id='123', artifact=[2, 8, 0, 6, 0, 0, 1, 5, 0, 0])

## 与模型结合使用

使用 [工具调用模型](/docs/how_to/tool_calling/)，我们可以轻松地使用模型调用工具并生成 ToolMessages：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>


In [5]:
# | echo: false
# | output: false

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

In [ ]:
llm_with_tools = llm.bind_tools([generate_random_ints])

ai_msg = llm_with_tools.invoke("生成 6 个小于 25 的正整数")
ai_msg.tool_calls

[{'name': 'generate_random_ints',
  'args': {'min': 1, 'max': 24, 'size': 6},
  'id': 'toolu_01EtALY3Wz1DVYhv1TLvZGvE',
  'type': 'tool_call'}]

In [7]:
generate_random_ints.invoke(ai_msg.tool_calls[0])

ToolMessage(content='Successfully generated array of 6 random ints in [1, 24].', name='generate_random_ints', tool_call_id='toolu_01EtALY3Wz1DVYhv1TLvZGvE', artifact=[2, 20, 23, 8, 1, 15])

如果我们仅传递工具调用参数，我们将只会得到内容部分：

In [8]:
generate_random_ints.invoke(ai_msg.tool_calls[0]["args"])

'Successfully generated array of 6 random ints in [1, 24].'

如果我们想声明式地创建一个链，可以这样做：

In [ ]:
from operator import attrgetter

chain = llm_with_tools | attrgetter("tool_calls") | generate_random_ints.map()

chain.invoke("给我一个 1 到 5 之间的随机数")

[ToolMessage(content='Successfully generated array of 1 random ints in [1, 5].', name='generate_random_ints', tool_call_id='toolu_01FwYhnkwDPJPbKdGq4ng6uD', artifact=[5])]

## 从 BaseTool 类创建

如果您希望直接创建一个 BaseTool 对象，而不是用 `@tool` 装饰一个函数，可以这样做：

In [ ]:
from langchain_core.tools import BaseTool


class GenerateRandomFloats(BaseTool):
    name: str = "generate_random_floats"
    description: str = "生成范围 [min, max] 内的 size 个随机浮点数。"
    response_format: str = "content_and_artifact"

    ndigits: int = 2

    def _run(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
        range_ = max - min
        array = [
            round(min + (range_ * random.random()), ndigits=self.ndigits)
            for _ in range(size)
        ]
        content = f"生成了 {size} 个浮点数，范围为 [{min}, {max}]，保留 {self.ndigits} 位小数。"
        return content, array

    # 可选地定义一个等效的异步方法

    # async def _arun(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
    #     ...

In [11]:
rand_gen = GenerateRandomFloats(ndigits=4)
rand_gen.invoke({"min": 0.1, "max": 3.3333, "size": 3})

'Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.'

In [12]:
rand_gen.invoke(
    {
        "name": "generate_random_floats",
        "args": {"min": 0.1, "max": 3.3333, "size": 3},
        "id": "123",
        "type": "tool_call",
    }
)

ToolMessage(content='Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.', name='generate_random_floats', tool_call_id='123', artifact=[1.5789, 2.464, 2.2719])